# Data Cleaning
This file is where I work with the sausage data to get it ready for my specific purposes. I will do this by
1. Loading the data from [thesausagedatabase](https://thesausagedatabase.com/)
2. Create a local backup of the database
3. 

In [140]:
import requests 
from datetime import datetime
import json
import pandas as pd


## Getting the Data
Again, I would like to place huge thanks to [The Sausage Database](https://thesausagedatabase.com/) for actually compiling all of this information. We are going to use requests to get the information from their website, and then save it to our files just in case. 

In [141]:
# URLS from The Sausage Database
sausages_url = "https://schatt3npakt.github.io/sausage-data/sausages.json"
nse_url = "https://schatt3npakt.github.io/sausage-data/nse.json"

# Get the response and json data for sausages
sausages_response = requests.get(sausages_url)
sausages_json = sausages_response.json()

# Use the response data to make a sausage ratings DataFrame
sausages_df = pd.DataFrame(sausages_json)

# Get the response and json data for non-sausage episodes (nse)
nse_response = requests.get(nse_url)
nse_json = nse_response.json()


#Use the response data to make a non-sausage ratings DataFrame
nse_df = pd.DataFrame(nse_json)

# Get todays date so that we know when the file was created
date_str = datetime.today().strftime('%Y-%m-%d')

# Save the sausage data to the appropriate location
with open(f'../data/raw/sausages_{date_str}.json', 'w') as f:
    json.dump(sausages_json, f)

# Save the non-sausage data to the appropriate location
with open(f'../data/raw/nse_{date_str}.json', 'w') as f:
    json.dump(nse_json, f)



___

## Sausage Episodes

The data in `sausages_df` covers traditional episodes of Oridnary Sausage. Some special ingredient or ingredients are used to create a specialty sausage. This often involves the ingredient(s) going through the sausage grinder with or without the addition of pork. We will see some other important parts of the process mentioned in our DataFrame. 

In [142]:
# See the most recent additions
sausages_df.tail()

,id,type,rating,dibl,dibu,song,episode,episodeID,episodeType,episodeLength
529,530,Snail Noodle Soup,3.5/5,5/5 Mark Ruffalos,0,Unknown Song,Snail Noodle Soup Sausage,6iyBn1_3O7o,Standard,05:50
530,531,Dry Iceage,0/5,Not Done,0,Creep,Snail Noodle Soup Sausage,eOYE6acGcR8,Standard,01:39
531,532,Cajun Andouille,4/5,3/5 Mark Ruffalos,0,Sunshine on My Shoulders,Cajun Andouille Sausage,hymBFQcK3b0,Standard,04:53
532,533,Sausage Skull,4/5,- Not Done -,0,- No Song -,Sausage Skull,bu5MEIOi_Vc,Standard,01:06
533,534,An Entire Bottle of Pure Vanilla Extract,0/5,4/5 Mark Buffalos,0,Light & Day,An Entire Bottle of Pure Vanilla Extract Sausage,OTPs_pZa5d4,Standard,06:04


### About the Columns
- `id`  is the index for each sausage
- `type` indicates what made the sausage special
- `rating` is how delicious the sausage is
- `dibl` is the ranking system for the *will it blow* segment. After sausage is extruded from the sausage machine there is some leftover in a small tube. Mr. Sausage aims this tube at a ceral box and sees if he can blow the debris free from the tube onto the box. It usually is rated between 0 and 5 Mark Ruffalos
- `dibu` measures if the sausage casing burst while cooking
- `song` is the song parody that Mr. Sausage sings while preparing the sausage. 
- `episode` is the youtube vidoes title
- `episodeID` is the youtube video id
- `episdeType` notes if there is any special theming for the episode such as "Youtube Comment Suggestions" and "Mrs. Sausage Makes a Sausage". 
- `episodeLength` is how long the episode is in `mm:ss` format. 

___

### Ratings
This is one of the most improtant ways of measuring sauasages. We want to make sure it is useable and well formatted.

In [143]:
sausages_df['rating'].value_counts()

rating
0/5             89
4/5             81
3/5             64
1/5             64
2/5             61
5/5             46
3.5/5           36
2.5/5           22
Disqualified    21
1.5/5           18
0.5/5           12
4.5/5           11
4(5              4
6/5              2
1(5              1
-1/5             1
-100/5           1
Name: count, dtype: int64

In [144]:
# Fix typo
sausages_df['rating'] = sausages_df['rating'].str.replace('(','/')

sausages_df['rating'].value_counts()

rating
0/5             89
4/5             85
1/5             65
3/5             64
2/5             61
5/5             46
3.5/5           36
2.5/5           22
Disqualified    21
1.5/5           18
0.5/5           12
4.5/5           11
6/5              2
-1/5             1
-100/5           1
Name: count, dtype: int64

### Data Cleaning Steps
1. Create a new column to indicate whether a sausage was disqualified.
2. Extract numeric scores for qualified sausages, removing labels like '2.5/5' and storing just the numeric value (e.g., 2.5).
3. Add a column to capture whether a sausage has been disqualified for easier analysis.

In [145]:
# Create a function to clean the rating column by extracting the numeric part
def clean_rating(value):
    if isinstance(value, str) and '/' in value:
        return value.split('/')[0]  # Extract the part before the slash
    else:
        return value  # Return the original value if no slash is found

# Apply the clean_rating function to create a cleaned 'rating' column
sausages_df['cleaned_rating'] = sausages_df['rating'].apply(clean_rating)

# Convert the cleaned rating to numeric values
sausages_df['rating_numeric'] = pd.to_numeric(sausages_df['cleaned_rating'], errors='coerce')



In [146]:
sausages_df['type'].value_counts()

type
Tiger Skin Chicken Feet                     2
New England Clam Chowder                    2
Frozen Potstickers                          1
Tofu                                        1
Beet and Goat Cheese                        1
                                           ..
Crab Cake                                   1
Mountain Dew and Doritos                    1
Lamb Kebabs and Stuffed Grape Leaves        1
Country Fried Steak                         1
An Entire Bottle of Pure Vanilla Extract    1
Name: count, Length: 532, dtype: int64

In [147]:
# uh oh, we have duplicates!
sausages_df[sausages_df['type'].isin(['Tiger Skin Chicken Feet', 'New England Clam Chowder'])]


,id,type,rating,dibl,dibu,song,episode,episodeID,episodeType,episodeLength,cleaned_rating,rating_numeric
40,41,New England Clam Chowder,Disqualified,Not Done,1,Islands In The Stream,New England Clam Chowder but as a Sausage,hrPuJIfFe9k,Standard,01:00,Disqualified,NaN
461,462,New England Clam Chowder,2.5/5,1/5 Mark Ruffalos,0,Thunder,New England Clam Chowder Sausage ReGrind,ZAUdLEofuVM,Standard,05:09,2.5,2.5
517,518,Tiger Skin Chicken Feet,5/5,5/5 Mark Ruffalos,0,Crazy World,Tiger Skin Chicken Feet Sausage,P4quUqmFS-4,Standard,06:23,5,5.0
518,519,Tiger Skin Chicken Feet,5/5,5/5 Mark Ruffalos,0,Crazy World,Tiger Skin Chicken Feet Sausage,P4quUqmFS-4,Standard,06:23,5,5.0


It appears that "Tiger Skin Chicken Feet" is mistakenly listed twice in the dataset. We'll remove the duplicate entry, unlike the "New England Clam Chowder Regrind," which we will keep.


In [148]:
sausages_df= sausages_df.drop(518)
sausages_df[sausages_df['type'].isin(['Tiger Skin Chicken Feet', 'New England Clam Chowder'])]

,id,type,rating,dibl,dibu,song,episode,episodeID,episodeType,episodeLength,cleaned_rating,rating_numeric
40,41,New England Clam Chowder,Disqualified,Not Done,1,Islands In The Stream,New England Clam Chowder but as a Sausage,hrPuJIfFe9k,Standard,01:00,Disqualified,NaN
461,462,New England Clam Chowder,2.5/5,1/5 Mark Ruffalos,0,Thunder,New England Clam Chowder Sausage ReGrind,ZAUdLEofuVM,Standard,05:09,2.5,2.5
517,518,Tiger Skin Chicken Feet,5/5,5/5 Mark Ruffalos,0,Crazy World,Tiger Skin Chicken Feet Sausage,P4quUqmFS-4,Standard,06:23,5,5.0


In [149]:
# Identify duplicates based on 'episodeID' column
duplicate_episode_ids = sausages_df[sausages_df.duplicated(subset='episodeID',keep=False)]

# View some of the duplicates based on the 'episodeID' column
duplicate_episode_ids.sort_values(by='episodeID').head()


,id,type,rating,dibl,dibu,song,episode,episodeID,episodeType,episodeLength,cleaned_rating,rating_numeric
53,54,Colby and Monterey Jack Cheese Sticks,0/5,Not Done,1,- No Song -,Just a Brick of Cheese Sausage,6XsRNwSh-N4,Standard,03:45,0,0.0
52,53,Brick of Extra Sharp Cheddar Cheese,0/5,Yes (No Rating),1,- No Song -,Just a Brick of Cheese Sausage,6XsRNwSh-N4,Standard,03:45,0,0.0
29,30,MRE: Rib-shaped BBQ Patties,Disqualified,Not Done,1,"Pure Imagination, Fly Me To The Moon, Wonderwall",MRE Sausage Livestream,8YqWMJVp8A0,Livestream Recording,60:31,Disqualified,NaN
30,31,MRE: White Chicken Chunks,0/5,Yes (No Rating),1,"Take On Me, Country Roads, Baby Shark, Africa,...",MRE Sausage Livestream,8YqWMJVp8A0,Livestream Recording,60:31,0,0.0
113,114,Cinnamon Roll (Ground),4/5,Not Done,1,- No Song -,Cinnamon Roll Sausage,9LblZcUQsqY,YouTube Comments Suggestions,01:00,4,4.0


The entries that share an episode `id` are all from episodes where more than one sausage was judged.

### Will it blow?
"Will It Blow?" is a segment where Mr. Sausage blows the remaining sausage filling from the sausage-making tube at a cereal box. The result is usually ranked between 0 and 5 Mark Ruffalos.


In [150]:
sausages_df['dibl'].value_counts()

dibl
Not Done                     100
1/5 Mark Ruffalos             86
2/5 Mark Ruffalos             67
3/5 Mark Ruffalos             66
4/5 Mark Ruffalos             49
5/5 Mark Ruffalos             47
Yes (No Rating)               36
0/5 Mark Ruffalos             29
0.5/5 Mark Ruffalos           21
No (No Rating)                13
1.5/5 Mark Ruffalos            3
1/5 Frank Oz’s                 2
4/5 Mark Buffalos              2
- Not Done -                   2
2.5/5 Mark Ruffalos            2
5/5 Ben Afflecks               1
No                             1
2(5 Mark Ruffalos              1
5/5 Makr Ruffalos              1
1/5 Mark Ruffalos*             1
1.5 and 3.5 Mark Ruffalos      1
1(5 Mark Ruffalos              1
6/5 Mark Ruffalos              1
Name: count, dtype: int64

In [151]:
# Check for missing values in 'dibl' column
missing_values = sausages_df['dibl'].isna().sum()
print(f"Missing values in 'dibl': {missing_values}")


Missing values in 'dibl': 0


In [152]:
# Deal with misc. typos
sausages_df['dibl'] = sausages_df['dibl'].astype(str)  # Convert the column to string type
sausages_df['dibl'] = sausages_df['dibl'].str.replace('(5', '/5')  # Fix the typo with parentheses
sausages_df['dibl'] = sausages_df['dibl'].str.replace('- Not Done -', 'Not Done')
sausages_df['dibl'] = sausages_df['dibl'].str.replace('Makr', 'Mark')  # Fix the typo
sausages_df.loc[sausages_df['dibl'] == 'No', 'dibl'] = 'No (No Rating)'  # Replace 'No' with 'No (No Rating)'


print(sausages_df['dibl'].value_counts())

dibl
Not Done                     102
1/5 Mark Ruffalos             87
2/5 Mark Ruffalos             68
3/5 Mark Ruffalos             66
4/5 Mark Ruffalos             49
5/5 Mark Ruffalos             48
Yes (No Rating)               36
0/5 Mark Ruffalos             29
0.5/5 Mark Ruffalos           21
No (No Rating)                14
1.5/5 Mark Ruffalos            3
2.5/5 Mark Ruffalos            2
1/5 Frank Oz’s                 2
4/5 Mark Buffalos              2
6/5 Mark Ruffalos              1
5/5 Ben Afflecks               1
1/5 Mark Ruffalos*             1
1.5 and 3.5 Mark Ruffalos      1
Name: count, dtype: int64


In [153]:
# For *one* of the episodes where mr. sausage tried two things they listed both in one row
sausages_df[sausages_df['dibl'] == '1.5 and 3.5 Mark Ruffalos'] 

,id,type,rating,dibl,dibu,song,episode,episodeID,episodeType,episodeLength,cleaned_rating,rating_numeric
412,413,Onion Rings,2.5/5,1.5 and 3.5 Mark Ruffalos,0,East Bound And Down,Nathan's Frozen Onion Rings Sausage,ejkPYg6jGqc,Standard,03:55,2.5,2.5


For the episode [Nathan's Frozen Onion Rings Sausage](https://www.youtube.com/watch?v=ejkPYg6jGqc) there is a sausage made from *only* frozen onion rings and one that is made with both pork and onion rings. After watching the episode I have the information for the results. Now we will create two entries with id 413 and 414 but  will need to update the `id` column for all following entries.

In [154]:
onion_ring_sausage = sausages_df[sausages_df['dibl'] == '1.5 and 3.5 Mark Ruffalos']

# create all onion ring row
onion_ring_no_pork = onion_ring_sausage.copy()
onion_ring_no_pork['type'] = 'Onion Rings (No Pork)'
onion_ring_no_pork['dibl'] = '1/5 Mark Ruffalos'
onion_ring_no_pork['rating'] = '1/5'
onion_ring_no_pork['rating_numeric'] = 1

# fix the actual row 
onion_ring_pork = onion_ring_sausage.copy()
onion_ring_pork['type'] = 'Onion Rings (With Pork)'
onion_ring_pork['dibl'] = '3/5 Mark Ruffalos'
onion_ring_pork['rating'] = '2/5'
onion_ring_pork['rating_numeric'] = 2

# remove the old row 
sausages_df = sausages_df[sausages_df['dibl'] != '1.5 and 3.5 Mark Ruffalos']

# combine the three dataframes
sausages_df = pd.concat([sausages_df, onion_ring_no_pork, onion_ring_pork])

# reset the index
sausages_df.reset_index(drop=True, inplace=True)

sausages_df[sausages_df['type'].str.contains('onion', case=False, na=False)]

,id,type,rating,dibl,dibu,song,episode,episodeID,episodeType,episodeLength,cleaned_rating,rating_numeric
16,17,Bloomin Onion,1/5,Not Done,1,Down Under,Bloomin Onion Sausage,tfNg_Wcm9Rc,Standard,06:15,1,1.0
106,107,Onion,0/5,5/5 Mark Ruffalos,1,Love Rollercoaster,Onion Sausage,eeIC6uJYo3I,YouTube Comments Suggestions,01:00,0,0.0
150,151,Liver and Onions,3/5,0.5/5 Mark Ruffalos,1,"Signed Sealed Delivered I'm Yours, Keep It Com...",Liver and Onions Sausage,u_pZrlv_IMw,Standard,04:20,3,3.0
338,339,Caramelized Onions,4/5,3/5 Mark Ruffalos,1,Hotel California,Caramelized Onions Sausage,f_Q3aQyfUQw,Standard,06:41,4,4.0
532,413,Onion Rings (No Pork),1/5,1/5 Mark Ruffalos,0,East Bound And Down,Nathan's Frozen Onion Rings Sausage,ejkPYg6jGqc,Standard,03:55,2.5,1.0
533,413,Onion Rings (With Pork),2/5,3/5 Mark Ruffalos,0,East Bound And Down,Nathan's Frozen Onion Rings Sausage,ejkPYg6jGqc,Standard,03:55,2.5,2.0


#### Mark Ruffalo Conversion System
As we have seen the *will it blow* segment is also a string we will need to convert to an integer. There are however several other isues. The first, are that some wibl segments are rated on a yes/no system instead of a Mark Ruffalo system. The second is that some of the wibl segments were rated with a different celebrity as the unit. If I was being strict, we would need to count those differently, but this is Sausage Math and sometimes we get to bend the rules. 

## Goals
We are going to create a cleaner `dibl` column. 

In [155]:
# copy the original data
sausages_df['dibl_copy'] = sausages_df['dibl']

# cleaning function
def clean_dibl(value):
    if '/' in value:
        return value.split('/')[0]
    elif 'Yes' in value:
        return 'Yes'
    elif 'No ' in value:
        return 'No'
    else:
        # otherwise if this block is run twice you lose all the numbers
        return value
    
# use our function to clean the dibl column
sausages_df['dibl'] = sausages_df['dibl'].apply(clean_dibl)

# check that we got what we wanted
sausages_df['dibl'].value_counts()


dibl
Not Done    102
1            91
2            68
3            67
4            51
5            49
Yes          36
0            29
0.5          21
No           14
1.5           3
2.5           2
6             1
Name: count, dtype: int64

In [156]:
# create a column of just the numeric ones 
sausages_df['dibl_numeric'] = pd.to_numeric(sausages_df['dibl'], errors = 'coerce')
sausages_df['dibl_numeric'].value_counts()


dibl_numeric
1.0    91
2.0    68
3.0    67
4.0    51
5.0    49
0.0    29
0.5    21
1.5     3
2.5     2
6.0     1
Name: count, dtype: int64

In [157]:
sausages_df.tail()

,id,type,rating,dibl,dibu,song,episode,episodeID,episodeType,episodeLength,cleaned_rating,rating_numeric,dibl_copy,dibl_numeric
529,532,Cajun Andouille,4/5,3,0,Sunshine on My Shoulders,Cajun Andouille Sausage,hymBFQcK3b0,Standard,04:53,4,4.0,3/5 Mark Ruffalos,3.0
530,533,Sausage Skull,4/5,Not Done,0,- No Song -,Sausage Skull,bu5MEIOi_Vc,Standard,01:06,4,4.0,Not Done,NaN
531,534,An Entire Bottle of Pure Vanilla Extract,0/5,4,0,Light & Day,An Entire Bottle of Pure Vanilla Extract Sausage,OTPs_pZa5d4,Standard,06:04,0,0.0,4/5 Mark Buffalos,4.0
532,413,Onion Rings (No Pork),1/5,1,0,East Bound And Down,Nathan's Frozen Onion Rings Sausage,ejkPYg6jGqc,Standard,03:55,2.5,1.0,1/5 Mark Ruffalos,1.0
533,413,Onion Rings (With Pork),2/5,3,0,East Bound And Down,Nathan's Frozen Onion Rings Sausage,ejkPYg6jGqc,Standard,03:55,2.5,2.0,3/5 Mark Ruffalos,3.0


In [158]:
# write to csv
sausages_df.to_csv('../data/processed/sausages_df.csv', index=False)